In [1]:
import pandas as pd
import mysql.connector
import getpass,os

from mysql_connect import MySqlConnection

import numpy as np
import pandas as pd
import os
from pathlib import Path


In [2]:

movie_df = pd.read_csv('movies.csv', encoding='ISO-8859-1', on_bad_lines='skip', sep=';')
movie_df.dropna(inplace=True)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3795 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      3795 non-null   object 
 1   rating    3795 non-null   object 
 2   genre     3795 non-null   object 
 3   year      3795 non-null   int64  
 4   released  3795 non-null   object 
 5   score     3795 non-null   float64
 6   votes     3795 non-null   float64
 7   director  3795 non-null   object 
 8   writer    3795 non-null   object 
 9   star      3795 non-null   object 
 10  country   3795 non-null   object 
 11  budget    3795 non-null   int64  
 12  gross     3795 non-null   float64
 13  company   3795 non-null   object 
 14  runtime   3795 non-null   float64
dtypes: float64(4), int64(2), object(9)
memory usage: 474.4+ KB


In [3]:
### Mysql Server Connection 

budget_schema_config = {
    "host":"127.0.0.1",
    "user":"analysis"

}

mysql_conn = MySqlConnection(**budget_schema_config)
mysql_conn.connect()

MySql Server Connection Established...

Connection Details:
User --> analysis@localhost
Date Time --> 2025-01-09 11:00:49 PM


In [4]:
mysql_conn.select_database(db_name='Campusx')

Connected to database: Campusx


In [5]:
mysql_conn.select_table_query('select * from marks')

,student_id,name,branch,marks
0,1,Nitish,EEE,82
1,2,Rishabh,EEE,91
2,3,Anukant,EEE,69
3,4,Rupesh,EEE,55
4,5,Shubham,CSE,78
5,6,Ved,CSE,43
6,7,Deepak,CSE,98
7,8,Arpan,CSE,95
8,9,Vinay,ECE,95
9,10,Ankit,ECE,88


In [6]:
## Min Max Avg Sum 
sql = """ 
select *,
min(marks) over(partition by branch) as Min,
max(marks) over(partition by branch) as Max,
avg(marks) over(partition by branch) as Avg,
sum(marks) over(partition by branch) as Sum
from marks
order by student_id
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,Min,Max,Avg,Sum
0,1,Nitish,EEE,82,55,91,74.2500,297
1,2,Rishabh,EEE,91,55,91,74.2500,297
2,3,Anukant,EEE,69,55,91,74.2500,297
3,4,Rupesh,EEE,55,55,91,74.2500,297
4,5,Shubham,CSE,78,43,98,78.5000,314
5,6,Ved,CSE,43,43,98,78.5000,314
6,7,Deepak,CSE,98,43,98,78.5000,314
7,8,Arpan,CSE,95,43,98,78.5000,314
8,9,Vinay,ECE,95,81,95,89.7500,359
9,10,Ankit,ECE,88,81,95,89.7500,359


In [7]:
## Rank 
sql = """ 
select *,
rank() over(partition by branch order by marks desc) 
from marks
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,rank() over(partition by branch order by marks desc)
0,7,Deepak,CSE,98,1
1,8,Arpan,CSE,95,2
2,5,Shubham,CSE,78,3
3,6,Ved,CSE,43,4
4,9,Vinay,ECE,95,1
5,12,Rohit,ECE,95,1
6,10,Ankit,ECE,88,3
7,11,Anand,ECE,81,4
8,2,Rishabh,EEE,91,1
9,1,Nitish,EEE,82,2


In [8]:
## Dense Rank 
sql = """ 
select *,
dense_rank() over(partition by branch order by marks desc) 
from marks
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,dense_rank() over(partition by branch order by marks desc)
0,7,Deepak,CSE,98,1
1,8,Arpan,CSE,95,2
2,5,Shubham,CSE,78,3
3,6,Ved,CSE,43,4
4,9,Vinay,ECE,95,1
5,12,Rohit,ECE,95,1
6,10,Ankit,ECE,88,2
7,11,Anand,ECE,81,3
8,2,Rishabh,EEE,91,1
9,1,Nitish,EEE,82,2


In [9]:
## Row Number
sql = """ 
select *,
row_number () over() as rnk
from marks
"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,rnk
0,1,Nitish,EEE,82,1
1,2,Rishabh,EEE,91,2
2,3,Anukant,EEE,69,3
3,4,Rupesh,EEE,55,4
4,5,Shubham,CSE,78,5
5,6,Ved,CSE,43,6
6,7,Deepak,CSE,98,7
7,8,Arpan,CSE,95,8
8,9,Vinay,ECE,95,9
9,10,Ankit,ECE,88,10


In [10]:
## Row Number partition by branch
sql = """ 
select *,
row_number() over(partition by branch) row_num
from marks;

"""
mysql_conn.select_table_query(sql)

,student_id,name,branch,marks,row_num
0,5,Shubham,CSE,78,1
1,6,Ved,CSE,43,2
2,7,Deepak,CSE,98,3
3,8,Arpan,CSE,95,4
4,9,Vinay,ECE,95,1
5,10,Ankit,ECE,88,2
6,11,Anand,ECE,81,3
7,12,Rohit,ECE,95,4
8,1,Nitish,EEE,82,1
9,2,Rishabh,EEE,91,2


# Creating Database and its respective tables

In [11]:
Path(os.getcwd())

WindowsPath('d:/ML_NOTE/SQL/Python_4_sql')

In [12]:
data_set_path = 'D:\ML_NOTE_DATASET\Analysis_dataset'
data_set_path = Path(data_set_path)
data_set_path

WindowsPath('D:/ML_NOTE_DATASET/Analysis_dataset')

In [13]:
zomato_df = pd.read_csv(f"{data_set_path}/zomato.csv", encoding='ISO-8859-1', on_bad_lines='skip')
zomato_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [14]:
mysql_conn.create_datebase(db_name='Zomato')

Database Zomato created successfully


In [15]:
mysql_conn.connect_to_database(db_name='Zomato')

Connected to database: Zomato


In [17]:
table_name='Zomato_table'
primary_key='Restaurant ID'

In [18]:
zomato_create_tbl_script = mysql_conn.create_tbl_script(df=zomato_df,table_name=table_name, primary_key=primary_key)

drop table if exists Zomato_table; 
CREATE TABLE Zomato_table (
Restaurant ID INTEGER(225) PRIMARY KEY AUTO_INCREMENT
,Restaurant Name VARCHAR(225)
,Country Code INTEGER(225)
,City VARCHAR(225)
,Address VARCHAR(225)
,Locality VARCHAR(225)
,Locality Verbose VARCHAR(225)
,Longitude DECIMAL(20, 2)
,Latitude DECIMAL(20, 2)
,Cuisines VARCHAR(225)
,Average Cost for two INTEGER(225)
,Currency VARCHAR(225)
,Has Table booking VARCHAR(225)
,Has Online delivery VARCHAR(225)
,Is delivering now VARCHAR(225)
,Switch to order menu VARCHAR(225)
,Price range INTEGER(225)
,Aggregate rating DECIMAL(20, 2)
,Rating color VARCHAR(225)
,Rating text VARCHAR(225)
,Votes INTEGER(225)
);


In [19]:
mysql_conn.create_table(zomato_df, table_name=table_name)

Error creating table: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
